In [ ]:
1

In [ ]:
import sys

sys.path.append("technique")
from utils import *

In [ ]:
import pandas as pd

In [ ]:
import culture.centre_choregraphique
import dee.activite
import dee.cantine
import mobilite.mobilite
import sports.sports

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
tbs = CountryTaxBenefitSystem()
tbs.load_extension("openfisca_france_local")

In [ ]:
from scenario import QfFiscalReform, StatutReform, CEReform

In [ ]:
qf_fiscal_reform = QfFiscalReform(tbs)
statut_reform = StatutReform(qf_fiscal_reform)
ce_reform = CEReform(statut_reform)
reform = ce_reform  # qf_fiscal_reform
reform = qf_fiscal_reform

In [ ]:
def ex(res):
    if type(res) == tuple:
        return res[0]
    else:
        return res

In [ ]:
sample_count = 1

In [ ]:
res = [
    culture.centre_choregraphique.get_results(tbs, sample_count, reform),
    dee.activite.get_results(tbs, sample_count, reform),
    dee.cantine.get_results(tbs, sample_count, reform),
    mobilite.mobilite.get_results(tbs, sample_count, reform),
    sports.sports.get_results(tbs, sample_count, reform),
]

In [ ]:
pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
full = pd.concat([ex(r) for r in res])

In [ ]:
totals = (
    full.groupby(("Direction", ""))
    .sum(numeric_only=True)
    .sort_values(("Recettes base", "Moyenne"), ascending=False)[
        [("Recettes base", "Moyenne")]
    ]
    .round()
    .reset_index()
)
totals.columns = pd.Index([("Direction", ""), ("Direction", "Total")])
totals

In [ ]:
writer = pd.ExcelWriter("Global_tables.xlsx")
totals.to_excel(writer, sheet_name="Direction Total")
writer.close()

In [ ]:
details = pd.merge(
    full, totals, left_on=[("Direction", "")], right_on=[("Direction", "")]
).sort_values([("Direction", "Total"), ("Recettes base", "Moyenne")], ascending=False)
main_details = details[
    [("Direction", ""), ("Service", ""), ("Quantité", ""), ("Recettes base", "Moyenne")]
]
main_details

In [ ]:
if reform:
    full[("Écart", "euros")] = (
        full[(("Recettes réforme", "Moyenne"))] - full[(("Recettes base", "Moyenne"))]
    )
    full[("Écart", "%")] = (
        full[(("Écart", "euros"))] / full[(("Recettes base", "Moyenne"))]
    )
    resume = (
        full.groupby(("Direction", ""))
        .sum(numeric_only=True)
        .sort_values(("Recettes base", "Moyenne"), ascending=False)
    )
    resume[("Écart", "%")] = (
        resume[(("Écart", "euros"))] / resume[(("Recettes base", "Moyenne"))]
    )
    resume

In [ ]:
resume

In [ ]:
from openfisca_core.parameters.parameter_scale import (
    ParameterScale,
    SingleAmountTaxScale,
)
from openfisca_core import parameters

In [ ]:
from baremes import baremes, build_table_data

In [ ]:
for b in baremes:
    assert type(baremes[b](tbs.parameters)) == ParameterScale
    assert (
        type(baremes[b](tbs.get_parameters_at_instant(base_period)))
        == SingleAmountTaxScale
    )

In [ ]:
len(baremes)

In [ ]:
if False:
    from dotenv import load_dotenv

    load_dotenv()
    import os
    import requests
    import json

    GRIST_DOC_ID = os.getenv("GRIST_DOC_ID")
    GRIST_API_KEY = os.getenv("GRIST_API_KEY")
    GRIST_SERVER = os.getenv("GRIST_SERVER")

    columns, records = build_table_data(tbs)
    res_table = requests.post(
        f"{GRIST_SERVER}/api/docs/{GRIST_DOC_ID}/tables",
        data=json.dumps({"tables": [{"id": "Tableau QF", "columns": columns}]}),
        headers={
            "Authorization": f"Bearer {GRIST_API_KEY}",
            "Content-Type": "application/json",
        },
    ).json()
    table_id = res_table["tables"][0]["id"]
    requests.post(
        f"{GRIST_SERVER}/api/docs/{GRIST_DOC_ID}/tables/{table_id}/records",
        data=json.dumps({"records": records}),
        headers={
            "Authorization": f"Bearer {GRIST_API_KEY}",
            "Content-Type": "application/json",
        },
    )
    res_records = requests.get(
        f"{GRIST_SERVER}/api/docs/{GRIST_DOC_ID}/tables/{table_id}/records",
        headers={
            "Authorization": f"Bearer {GRIST_API_KEY}",
        },
    ).json()
    res_records

In [ ]:
df, compens_constant = mobilite.mobilite.get_df()
rdf = pd.DataFrame(
    data={
        "pu_fichier": df.PU,
        "pu_calc": df.PU * 0,
        "tp": df.PU * 0,
    }
)
data, complement_df = mobilite.mobilite.build_data(df, rdf)

In [ ]:
btbs = CountryTaxBenefitSystem()
btbs.load_extension("openfisca_france_local")

In [ ]:
b_scenario = StrasbourgSurveyScenario(btbs, data=data)
(b_res, b_recettes) = mobilite.mobilite.compute_result(
    b_scenario, complement_df, 0, compens_constant
)

In [ ]:
btbs = CountryTaxBenefitSystem()
btbs.load_extension("openfisca_france_local")
b_scenario = StrasbourgSurveyScenario(btbs, data=data)
(b_res, b_recettes) = mobilite.mobilite.compute_result(
    b_scenario, complement_df, 0, compens_constant
)

In [ ]:
rtbs = gristSimulationReform(btbs)

In [ ]:
r_scenario = StrasbourgSurveyScenario(rtbs, data=data)
(r_res, r_recettes) = mobilite.mobilite.compute_result(
    r_scenario, complement_df, 0, compens_constant
)

In [ ]:
comp_res = b_res.merge(
    r_res, left_index=True, right_index=True, suffixes=["_pre", "_post"]
)

In [ ]:
full.__dict__

In [ ]:
comp_res.sample_id_pre.unique()

In [ ]:
pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
pd.pivot_table(
    comp_res,
    index="pu_calc_ht_pre",
    columns="pu_calc_ht_post",
    values="sample_id_pre",
    aggfunc=len,
    fill_value=0,
)

In [ ]:
b_res.groupby("sample_id").pu_calc_ht.sum(numeric_only=True)